In [4]:
import numpy as np

import pandas as pd

In [6]:
df_train_vardaan = pd.read_csv(r'C:\Users\varda\OneDrive\Desktop\nlp_proj\dataset\train_vardaan.csv')

In [7]:
df_train_vardaan = df_train_vardaan.dropna()
df_train_vardaan["combined"] = (
    "Consumer complaint: " + df_train_vardaan["Consumer complaint narrative"].str.strip() 
)

In [8]:
df_train_vardaan

,Issue,Consumer complaint narrative,Company public response,Complaint ID,combined
0,Struggling to pay mortgage,I'm filing this complaint today to bring to yo...,Company has responded to the consumer and the ...,5162002,Consumer complaint: I'm filing this complaint ...
2,Incorrect information on credit report,The XXXX XXXX account being reported on my cre...,Company chooses not to provide a public response,1502891,Consumer complaint: The XXXX XXXX account bein...
4,Managing the loan or lease,For starters American Credit Acceptance is say...,Company believes it acted appropriately as aut...,6382987,Consumer complaint: For starters American Cred...
7,Payoff process,Synchrony Bank did NOT adequately remind me th...,Company has responded to the consumer and the ...,1894570,Consumer complaint: Synchrony Bank did NOT ade...
11,Getting a credit card,15 USC 1601 I am writing to delete the followi...,Company has responded to the consumer and the ...,6684260,Consumer complaint: 15 USC 1601 I am writing t...
...,...,...,...,...,...
2489,Can't stop charges to bank account,XXXX XXXX XXXX in XXXX XXXX was charged severa...,Company chooses not to provide a public response,1712560,Consumer complaint: XXXX XXXX XXXX in XXXX XXX...
2490,Struggling to pay mortgage,We received a modification offer but sent appe...,Company has responded to the consumer and the ...,5962480,Consumer complaint: We received a modification...
2491,Can't stop charges to bank account,I had contacted a payday loan debt consolidati...,Company believes it acted appropriately as aut...,1506770,Consumer complaint: I had contacted a payday l...
2492,Incorrect information on your report,I am a victim of identity theft and I have fil...,Company has responded to the consumer and the ...,5494202,Consumer complaint: I am a victim of identity ...


In [9]:
bf=df_train_vardaan.drop(['Issue','Consumer complaint narrative','Company public response'], axis=1)

In [10]:
bf

,Complaint ID,combined
0,5162002,Consumer complaint: I'm filing this complaint ...
2,1502891,Consumer complaint: The XXXX XXXX account bein...
4,6382987,Consumer complaint: For starters American Cred...
7,1894570,Consumer complaint: Synchrony Bank did NOT ade...
11,6684260,Consumer complaint: 15 USC 1601 I am writing t...
...,...,...
2489,1712560,Consumer complaint: XXXX XXXX XXXX in XXXX XXX...
2490,5962480,Consumer complaint: We received a modification...
2491,1506770,Consumer complaint: I had contacted a payday l...
2492,5494202,Consumer complaint: I am a victim of identity ...


In [11]:
!pip install -U sentence-transformers

!pip install torch==1.2.0 torchvision==0.4.0 -f

  Obtaining dependency information for sentence-transformers from https://files.pythonhosted.org/packages/06/97/57afa3d05801b6b9305f96a7ce5995e12c1d2ba25ce66747de107816b0b5/sentence_transformers-2.3.1-py3-none-any.whl.metadata
  Obtaining dependency information for torch>=1.11.0 from https://files.pythonhosted.org/packages/58/b8/51b956c2da9729390a3080397cd2f31171394543af7746681466e372f69a/torch-2.2.0-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/977.5 kB ? eta -:--:--
     ----------- -------------------------- 286.7/977.5 kB 5.9 MB/s eta 0:00:01
     ------------------------------------  972.8/977.5 kB 12.3 MB/s eta 0:00:01
     ------------------------------------- 977.5/977.5 kB 10.3 MB/s eta 0:00:00
  Obtaining dependency information for typing-extensions>=3.7.4.3 from https://files.pythonhosted.org/packages/b7/f4/6a90020cd2d93349b442bfcb657d0dc91eee65491600b2cb1d388bc98e6b/typing_extensions-4.9.0-py3-none-any.whl.metadata
   -----------------


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

-f option requires 1 argument


In [12]:
from transformers import BertTokenizer, BertModel
import torch
import pandas as pd

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'  # You can choose a different pre-trained model
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# # Example DataFrame with a "complaint" column
# df = pd.DataFrame({"complaint": ["This is an example sentence for word embedding using BERT.",
#                                 "Another complaint about a different issue."]})

# Function to get BERT embeddings for a sentence
def get_bert_embeddings(sentence):
    # Tokenize and truncate/pad the sentence to fit within the model's maximum sequence length
    tokens = tokenizer(sentence, truncation=True, padding=True, return_tensors="pt")
    
    # Get BERT embeddings
    with torch.no_grad():
        outputs = model(**tokens)
    
    embeddings = outputs.last_hidden_state[:, 0, :]
    return embeddings.numpy()

# Apply the function to the "complaint" column
bf['bert_embeddings'] = bf['combined'].apply(get_bert_embeddings)

# Display the DataFrame with BERT embeddings
print(bf)


C:\Users\varda\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


C:\Users\varda\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\varda\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


      Complaint ID                                           combined  \
0          5162002  Consumer complaint: I'm filing this complaint ...   
2          1502891  Consumer complaint: The XXXX XXXX account bein...   
4          6382987  Consumer complaint: For starters American Cred...   
7          1894570  Consumer complaint: Synchrony Bank did NOT ade...   
11         6684260  Consumer complaint: 15 USC 1601 I am writing t...   
...            ...                                                ...   
2489       1712560  Consumer complaint: XXXX XXXX XXXX in XXXX XXX...   
2490       5962480  Consumer complaint: We received a modification...   
2491       1506770  Consumer complaint: I had contacted a payday l...   
2492       5494202  Consumer complaint: I am a victim of identity ...   
2495       1837677  Consumer complaint: I am a victim of identity ...   

                                        bert_embeddings  
0     [[-0.5966147, -0.4182573, 0.101597264, 0.25041...  
2     [

In [16]:
bf[["bert_embeddings"]].to_csv(r"C:\Users\varda\OneDrive\Desktop\nlp_proj\dataset\Embeddings.csv")